Start by defining a `Python` function that we want to compute.

In [1]:
def f(a, b, c):
    return (a & b) ^ c

Generate a circuit that computes this function. To implement the logical operations we use  standard verilog gates, which are available in `mantle.verilog.gates`.

In [2]:
import magma as m
from mantle.verilog.gates import And, XOr

main = m.DefineCircuit('verilator_example', "a", m.In(m.Bit), "b", m.In(m.Bit), "c", m.In(m.Bit), "d", m.Out(m.Bit))
t = And(2)(main.a, main.b)
d = XOr(2)(t, main.c)
m.wire(d, main.d)
m.EndCircuit()

m.compile("build/verilator_example", main)
%cat build/verilator_example.v

import verilog
compiling verilator_example
module verilator_example (input  a, input  b, input  c, output  d);
wire  inst0_0;
wire  inst1_0;
and inst0 (inst0_0, a, b);
xor inst1 (inst1_0, inst0_0, c);
assign d = inst1_0;
endmodule



Next, generate a verilator test harness in `C++` for the circuit. The test vectors are generated using the python function `f`. The verilator test bench compares the output of the simulator to those test vectors.

In [3]:
from magma.testing.verilator import compile as compileverilator

compileverilator('build/sim_verilator_example.cpp', main, f)
%cat build/sim_verilator_example.cpp

#include "Vverilator_example.h"
#include "verilated.h"
#include <cassert>
#include <iostream>

int main(int argc, char **argv, char **env) {
    Verilated::commandArgs(argc, argv);
    Vverilator_example* top = new Vverilator_example;

    unsigned int tests[8][4] = {
        { 0b0, 0b0, 0b0, 0b0 }, 
        { 0b0, 0b0, 0b1, 0b1 }, 
        { 0b0, 0b1, 0b0, 0b0 }, 
        { 0b0, 0b1, 0b1, 0b1 }, 
        { 0b1, 0b0, 0b0, 0b0 }, 
        { 0b1, 0b0, 0b1, 0b1 }, 
        { 0b1, 0b1, 0b0, 0b1 }, 
        { 0b1, 0b1, 0b1, 0b0 }, 
    };

    for(int i = 0; i < 8; i++) {
        unsigned int* test = tests[i];
        top->a = test[0];
        top->b = test[1];
        top->c = test[2];
        top->eval();
        assert(top->d == test[3]);
    }

    delete top;
    std::cout << "Success" << std::endl;
    exit(0);
}

Compile and run the test bench using `verilator`.

In [4]:
%%bash
cd build
verilator -Wall -Wno-DECLFILENAME --cc verilator_example.v --exe sim_verilator_example.cpp
make -C obj_dir -j -f Vverilator_example.mk Vverilator_example
./obj_dir/Vverilator_example

/Users/hanrahan/git/magmathon/notebooks/advanced/build
clang++  -I.  -MMD -I/usr/local/Cellar/verilator/3.924/share/verilator/include -I/usr/local/Cellar/verilator/3.924/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=0 -faligned-new -fbracket-depth=4096 -Qunused-arguments -Wno-parentheses-equality -Wno-sign-compare -Wno-uninitialized -Wno-unused-parameter -Wno-unused-variable -Wno-shadow       -c -o sim_verilator_example.o ../sim_verilator_example.cpp
/usr/bin/perl /usr/local/Cellar/verilator/3.924/share/verilator/bin/verilator_includer -DVL_INCLUDE_OPT=include Vverilator_example.cpp > Vverilator_example__ALLcls.cpp
/usr/bin/perl /usr/local/Cellar/verilator/3.924/share/verilator/bin/verilator_includer -DVL_INCLUDE_OPT=include Vverilator_example__Syms.cpp > Vverilator_example__ALLsup.cpp
clang++  -I.  -MMD -I/usr/local/Cellar/verilator/3.924/share/verilator/include -I/usr/local/Cellar/verilator/3.924/share/verilator/include/vltstd -DVL_PRINTF=prin